<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Web Scraping

_Author: Dave Yerrington (SF)_

---

## Before Class

#### Install Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`


#### Install GeckoDriver

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

#### Install Firefox

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

## Learning Objectives
- Revisit how to locate elements on a webpage
- Aquire unstructure data from the internet using Beautiful soup.
- Discuss limitations associated with simple requests and urllib libraries
- Introduce Selenium as a solution, and implement a scraper using selenium

## Lesson Guide

- [Introduction](#intro)
- [Building a web scraper](#building-scraper)
- [Retrieving data from the HTML page](#retrieving-data)
    - [Retrieving the restaurant names](#retrieving-names)
    - [Challenge: Retrieving the restaurant locations](#retrieving-locations)
    - [Retrieving the restaurant prices](#retrieving-prices)
    - [Retrieving the restaurant number of bookings](#retrieving-bookings)


- [Introducting Selenium](#selenium)
    - [Running JavaScript before scraping](#selenium-js)
    - [Using regex to only get digits](#selenium-regex)
    - [Challenge: Use Pandas to create a DataFrame of bookings](#challenge-pandas)
    - [Auto-typing using Selenium](#selenium-typing)


- [Summary](#summary)

<a id="intro"></a>
## Introduction

In this codealong lesson, we'll build a web scraper using urllib and BeautifulSoup. We will also explore how to use a headless browser called Selenium.

We'll begin by scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, and how many people booked it today.**

OpenTable provides all of this information on this given page: http://www.opentable.com/washington-dc-restaurant-listings

Let's inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

---

<a id="building-scraper"></a>
## Building a web scraper

Now, let's build a web scraper for OpenTable using urllib and Beautiful Soup:

In [2]:
# import our necessary first packages
from bs4 import BeautifulSoup
import urllib

In [3]:
# set the url we want to visit
url = "http://www.opentable.com/washington-dc-restaurant-listings"

# visit that url, and grab the html of said page
html = urllib.urlopen(url).read()

At this point, what is in html?

In [4]:
html[:500]

'          <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"/><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE"/> <title>Washington, D.C. Area Restaurants List | OpenTable</title>  <meta  name="description" content="Find Washington, D.C. Area restaurants. Search by location, cuisine, or price to refine restaurant results in the Washington, D.C. Area area." > </meta>  <meta  name="robots" content="noindex" > </meta>    <link rel="shortcut icon" href="//components.otst'

We will need to convert this html objct into a soup object so we can parse it using python and BS4

In [5]:
# convert this into a soup object
soup = BeautifulSoup(html, 'html.parser', from_encoding="utf-8")

<a id="retrieving-data"></a>
### Retrieving data from the HTML page

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? (Hint: We need to know where in the **HTML** the restaurant element is housed.) In order to find the HTML that renders the restaurant location, we can use Google Chrome's Inspect tool:

> http://www.opentable.com/washington-dc-restaurant-listings

> 1. Visit the URL above. 

> 2. Right-click on an element you are interested in, then choose Inspect (in Chrome). 

> 3. This will open the Developer Tools and show the HTML used to render the selected page element. 

> Throughout this lesson, we will use this method to find tags associated with elements of the page we want to scrape.

See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [7]:
# print the restaurant names
soup.find_all(name='span', attrs={'class':'rest-row-name-text'})

[<span class="rest-row-name-text">Ruffino's - Arlington</span>,
 <span class="rest-row-name-text">Joe's Place Pizza and Pasta</span>,
 <span class="rest-row-name-text">Rasika West End</span>,
 <span class="rest-row-name-text">Blue Duck Tavern</span>,
 <span class="rest-row-name-text">Farmers Fishers Bakers</span>,
 <span class="rest-row-name-text">Founding Farmers - Tysons</span>,
 <span class="rest-row-name-text">Founding Farmers - DC</span>,
 <span class="rest-row-name-text">Filomena Ristorante</span>,
 <span class="rest-row-name-text">District Commons</span>,
 <span class="rest-row-name-text">Ambar - Arlington</span>,
 <span class="rest-row-name-text">Eddie V's - Tysons Corner</span>,
 <span class="rest-row-name-text">Clyde's of Georgetown</span>,
 <span class="rest-row-name-text">Chez Billy Sud</span>,
 <span class="rest-row-name-text">Fogo de Chao Brazilian Steakhouse - Tyson's</span>,
 <span class="rest-row-name-text">Seasons 52 - Tysons Corner</span>,
 <span class="rest-row-name

It is important to always keep in mind the data types that were returned. Note this is a `list`, and we know that immediately by observing the outer square brackets and commas separating each tag.

Next, note the elements of the list are `Tag` objects, not strings. (If they were strings, they would be surrounded by quotes.) The Beautiful Soup authors chose to display a `Tag` object visually as a text representation of the tag and its contents. However, being an object, it has many methods that we can call on it. For example, next we will use the `encode_contents()` method to return the tag's contents encoded as a Python string.

<a id="retrieving-names"></a>
#### Retrieving the restaurant names

Now that we found a list of tags containing the restaurant names, let's think how we can loop through them all one-by-one. In the following cell, we'll print out the name (and **only** the clean name, not the rest of the html) of each restaurant.

In [8]:
# for each element you find, print out the restaurant name
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print entry.encode_contents()

Ruffino's - Arlington
Joe's Place Pizza and Pasta
Rasika West End
Blue Duck Tavern
Farmers Fishers Bakers
Founding Farmers - Tysons
Founding Farmers - DC
Filomena Ristorante
District Commons
Ambar - Arlington
Eddie V's - Tysons Corner
Clyde's of Georgetown
Chez Billy Sud
Fogo de Chao Brazilian Steakhouse - Tyson's
Seasons 52 - Tysons Corner
Sugar Factory - Pentagon Mall
J. Gilbert’s – Wood Fired Steaks &amp; Seafood - McLean
CIRCA at Foggy Bottom
Boqueria - DC
Tupelo Honey - Arlington
Nostos Restaurant
2941 Restaurant
Et Voila
Barrel &amp; Bushel
Texas Jacks Barbecue
The Capital Grille - Tysons Corner
Il Canale
Ray's The Steaks
BlackSalt
Sette Osteria
Palette 22
Clyde's of Chevy Chase
Chef Geoff's Tysons
Barley Mac
Medium Rare - Cleveland Park
Lyon Hall
The Palm Tysons Corner
Chima Steakhouse - Washington
Copperwood Tavern
Grillfish DC
Tony &amp; Joe's Seafood Place
La Chaumiere
Sequoia
The Palm Washington DC
The Capital Grille - Chevy Chase
CIRCA at Dupont
Clyde's at Mark Center
CIRCA

Great!

<a id="retrieving-locations"></a>
#### Challenge: Retrieving the restaurant locations

Can you repeat that process for finding the location? For example, barmini by Jose Andres is in the location listed as "Penn Quarter" in our search results.

In [18]:
for entry in soup.find_all(name='span', attrs={'class':'rest-row-meta--location rest-row-meta-text'}):
    print entry.encode_contents()

Arlington
Arlington
West End
West End
Georgetown
Tysons Corner / McLean
Foggy Bottom
Georgetown
Foggy Bottom
Arlington
Tysons Corner / McLean
Georgetown
Georgetown
Tysons Corner / McLean
Tysons Corner / McLean
Arlington
McLean
Foggy Bottom
Dupont Circle
Arlington
Vienna
Falls Church
Palisades Northwest
Tysons Corner / McLean
Arlington
Tysons Corner / McLean
Georgetown
Arlington
Palisades Northwest
Dupont Circle
Arlington
Friendship Heights
Tysons Corner / McLean
Arlington
Cleveland Park
Arlington
Tysons Corner / McLean
Vienna
Arlington
Dupont Circle
Georgetown
Georgetown
Georgetown
Dupont Circle
Chevy Chase
Dupont Circle
Alexandria
Arlington
Bethesda / Chevy Chase
Friendship Heights
Georgetown
Georgetown
West End
Tysons Corner / McLean
Georgetown
Arlington
Dupont Circle
Cleveland Park
Georgetown
Georgetown
Bethesda / Chevy Chase
Downtown
Arlington
Georgetown
Arlington
Arlington
Arlington
Georgetown
Cleveland Park
Foggy Bottom
Dupont Circle
Tysons Corner / McLean
Foggy Bottom
Georgetown

In [15]:
# first, see if you can identify the location for all elements -- print it out
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    content = entry.encode_contents()
    if ' - ' in content:
        print content[content.find(' - ')+3:]

Arlington
Tysons
DC
Arlington
Tysons Corner
Tyson's
Tysons Corner
Pentagon Mall
McLean
DC
Arlington
Tysons Corner
Cleveland Park
Washington
Chevy Chase
Chevy Chase
Tysons
Arlington VA
Cathedral Commons
Georgetown
Chevy Chase
Pentagon City
Arlington/ Ballston
Georgetown
Tysons Corner
Cleveland Park
Four Seasons Washington DC
DC
Georgetown
DC - Connecticut NW
McLean - Tysons Corner
Tysons Corner
Washington DC
Ballston
Arlington
DC
Georgetown


In [74]:
# now print out EACH location for the restaurants
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    content = entry.encode_contents()
    if ' - ' in content:
        print content[content.find(' - ')+3:]

<a id="retrieving-prices"></a>
#### Retrieving the restaurant prices

Ok, we've figured out the restaurant name and location. Now we need to grab the price (number of dollar signs on a scale of one to four) for each restaurant. We'll follow the same process.

In [19]:
# print out all prices
soup.find_all('div', {'class':'rest-row-pricing'})

[<div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $    $    </i>   $        </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pricing"><i>  $    $      </i>   $    $      </div>,
 <div class="rest-row-pri

In [20]:
# print out EACH number of dollar signs per restaurant
# this one is trickier to eliminate the html. Hint: try a nested find
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print entry.find('i').encode_contents()

  $    $      
  $    $      
  $    $    $    
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $      
  $    $      
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $    $    $  
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    
  $    $    $    
  $    $    $    
  $    $    $    
  $    $      
  $    $      
  $    $      
  $    $      
  $    $    $    $  
  $    $  

That looks great, but what if I wanted just the number of dollar signs per restaurant? Can you figure out a way to simply print out the number of dollar signs per restaurant listed?

In [24]:
# print the number of dollars signs per restaurant
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    print entry.find('i').encode_contents().count('$')

2
2
3
3
2
2
2
3
2
2
3
2
2
4
2
2
3
2
3
2
2
2
2
2
2
4
2
2
3
2
2
2
2
2
2
2
4
4
2
2
2
2
2
4
4
2
2
2
2
2
2
2
3
3
3
3
2
2
2
2
4
3
2
4
2
3
2
2
2
4
3
2
2
3
3
2
2
2
4
3
2
2
2
2
3
4
3
3
4
2
2
2
2
2
2
2
2
4
2
2


In [37]:
# take the name, location, price -> dataframe

name = []
for entry in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    name.append(entry.encode_contents())
                
location = []
for entry in soup.find_all(name='span', attrs={'class':'rest-row-meta--location rest-row-meta-text'}):
    location.append(entry.encode_contents())
                
price = []   
for entry in soup.find_all('div', {'class':'rest-row-pricing'}):
    price.append(entry.find('i').encode_contents().count('$'))


In [43]:
import pandas as pd
df = pd.DataFrame({'name': name, 'location':location, 'price':price})
df

,location,name,price
0,Arlington,Ruffino's - Arlington,2
1,Arlington,Joe's Place Pizza and Pasta,2
2,West End,Rasika West End,3
3,West End,Blue Duck Tavern,3
4,Georgetown,Farmers Fishers Bakers,2
5,Tysons Corner / McLean,Founding Farmers - Tysons,2
6,Foggy Bottom,Founding Farmers - DC,2
7,Georgetown,Filomena Ristorante,3
8,Foggy Bottom,District Commons,2
9,Arlington,Ambar - Arlington,2


Phew, nice work. 

<a id="retrieving-bookings"></a>
#### Retrieving the restaurant number of bookings

One more, right? We only need to find the number times a restaurant was booked. In the next cell, print out all objects that contain the number of times the restaurant was booked.

In [24]:
# print out all objects that contain the number of times the restaurant was booked
soup.find_all('div', {'class':'booking'})

[]

That's weird -- an empty set. Did we find the wrong element? What's going on here? Discuss.

How can we debug this? Any ideas?

In [30]:
# let's first try printing out all 'div' objects
#  NOTE: This is a too many objects to store in this notebook!
#        So, uncomment the code below to run it.

# for entry in soup.find_all('div'):
#     print entry

I still don't see it. Let's search our entire soup object:

In [31]:
# print out soup, do command+f for "booked ". 
#   Uncomment the below to run.

# soup

What do you notice? Why is this happening?

<a id="selenium"></a>
## Introducing Selenium

Selenium is a headless browser. It allows us to render JavaScript just as a human-navigated browser would.

To install Selenium, use one of the following:
- **Anaconda:** `conda install -c conda-forge selenium`
- **pip:** `pip install selenium`

You will also need GeckoDriver (this assumes you are using Homebrew for Mac): 

- ```brew install geckodriver```

Additionally, you will need to have downloaded the [Firefox browser](https://www.mozilla.org/en-US/firefox/new/?utm_source=google&utm_medium=cpc&utm_campaign=Firefox-Brand-US-GGL-Exact&utm_term=firefox&utm_content=A144_A203_A006336&gclid=Cj0KEQjwnPLKBRC-j7nt1b7OlZwBEiQAv8lMLJUyReT6cPzSYdmEA6uD3YDoieuuuusddgAU7XH6smEaAoje8P8HAQ&gclsrc=aw.ds) for the application in this lesson.

In [34]:
# import
from selenium import webdriver

Selenium requires us to determine a default browser to run. I'm going to opt for Firefox, but Chromium is also a very common choice. http://selenium-python.readthedocs.io/faq.html

In [35]:
# STOP
# what is going to happen when I run the next cell?

In [38]:
# create a driver called Firefox
driver = webdriver.Firefox()

Pretty crazy, right? Let's close that driver.

In [39]:
# close it
driver.close()

In [40]:
# let's boot it up, and visit a URL of our choice
driver = webdriver.Firefox()
driver.get("http://www.python.org")

Awesome. Now we're getting somewhere: programmatically controlling our browser like a human.

Let's return to our problem at hand. We need to visit the OpenTable listing for DC. Once there, we need to get the html to load. In the next cell, prove you can programmatically visit the page.

In [41]:
# visit our OpenTable page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# always good to check we've got the page we think we do
assert "OpenTable" in driver.title

In [42]:
driver.title

u'Washington, D.C. Area Restaurants List | OpenTable'

In [43]:
driver.close()

<a id="selenium-js"></a>
### Running JavaScript before scraping

Now, to resolve our JavaScript problem, there's a few things we can do. What I'll do in this case is request that the page load, wait one second, and then I'm going to grab the source html from the page. Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

In [44]:
# import sleep
from time import sleep

In [45]:
# visit our relevant page
driver = webdriver.Firefox()
driver.get("http://www.opentable.com/washington-dc-restaurant-listings")
# wait one second
sleep(1)
#grab the page source
html = driver.page_source

**Pop Quiz:** What do we need to do with this HTML?

In [47]:
# BeautifulSoup it!
html = BeautifulSoup(html, "lxml")

Now, let's return to our earlier problem: How do we locate bookings on the page?

In [48]:
# print out the number bookings for all restaurants
html.find_all('div', {'class':'booking'})

[<div class="booking"><span class="tadpole"></span>Booked 29 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 181 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 45 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 69 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 89 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 283 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 147 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 178 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 309 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 332 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 30 times today</div>,
 <div class="booking"><span class="tadpole"></span>Booked 80 times today</div>,
 <div class="booking"><span class=

In [49]:
# now print out each booking for the listings using a loop
for entry in html.find_all('div', {'class':'booking'}):
    print entry

<div class="booking"><span class="tadpole"></span>Booked 29 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 181 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 45 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 69 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 89 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 283 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 147 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 178 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 309 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 332 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 30 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 80 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 5

Let's grab just the text of each of these entries.

In [50]:
# do the same as above, but grabbing only the text content
for entry in html.find_all('div', {'class':'booking'}):
    print entry.text

Booked 29 times today
Booked 181 times today
Booked 45 times today
Booked 69 times today
Booked 89 times today
Booked 283 times today
Booked 147 times today
Booked 178 times today
Booked 309 times today
Booked 332 times today
Booked 30 times today
Booked 80 times today
Booked 53 times today
Booked 31 times today
Booked 56 times today
Booked 572 times today
Booked 42 times today
Booked 81 times today
Booked 30 times today
Booked 43 times today
Booked 29 times today
Booked 147 times today
Booked 71 times today
Booked 21 times today
Booked 195 times today
Booked 85 times today
Booked 43 times today
Booked 36 times today
Booked 29 times today
Booked 33 times today
Booked 37 times today
Booked 35 times today
Booked 206 times today
Booked 85 times today
Booked 34 times today
Booked 22 times today
Booked 51 times today
Booked 79 times today
Booked 101 times today
Booked 58 times today
Booked 29 times today
Booked 91 times today
Booked 78 times today
Booked 221 times today
Booked 265 times tod

In [51]:
driver.close()

We've succeeded!

<a id="selenium-regex"></a>
### Using regex to only get digits

But we can clean this up a little bit. We're going to use regular expressions (regex) to grab only the digits that are available in each of the text.

The best way to get good at regex is to, well, just keep trying and testing: http://pythex.org/

In [52]:
# import regex
import re

Given we haven't covered regex, I'll show you how to use the search function to match any given digit.

In [53]:
# for each entry, grab the text
for booking in html.find_all('div', {'class':'booking'}):
    # match all digits
    match = re.search(r'\d+', booking.text)
    
    if match:
        # print if found
        print match.group()
    else:
        # otherwise pass
        pass

29
181
45
69
89
283
147
178
309
332
30
80
53
31
56
572
42
81
30
43
29
147
71
21
195
85
43
36
29
33
37
35
206
85
34
22
51
79
101
58
29
91
78
221
265
33
18
49
24
40
58
24
31
27
35
265
39
353
35
30
12
12
30
62
92
44
8
50
40
55
15
22
99
30
24
13
16
13
82
17
68
14
50
34
16
28
15
30
30
21
84
12
37
88
14
29
12
16
64


Before we demonstrate all the other amazing things about headless browsers, let's finish up collecting the data we want from this current example. Do you suppose the html parsing we wrote above will still work on the page source we've grabbed from our headless browser?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [55]:
# print out all entries
#   NOTE: Has many entries. Uncomment the below code to run it!

# soup.find_all('div', {'class':'result content-section-list-row cf with-times'})

Look over the page. Does every single entry have each element we're seeking?
> I did this previously. I know for a fact that not every element has a number of recent bookings. That's probably exactly why OpenTable houses this in JavaScript: they want to continously update the number of bookings with the most relevant number of values.

In [56]:
# What happens when a booking is not available?
# Print out each booking entry, using the identification code we wrote above

for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    print entry.find('div', {'class':'booking'})

<div class="booking"><span class="tadpole"></span>Booked 29 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 181 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 45 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 69 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 89 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 283 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 147 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 178 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 309 times today</div>
None
<div class="booking"><span class="tadpole"></span>Booked 332 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 30 times today</div>
<div class="booking"><span class="tadpole"></span>Booked 80 times today</div>
<div class="booking"><span class="tadpole"></span>Boo

In [57]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
        print entry.find('div', {'class':'booking'}).text

Booked 29 times today
Booked 181 times today
Booked 45 times today
Booked 69 times today
Booked 89 times today
Booked 283 times today
Booked 147 times today
Booked 178 times today
Booked 309 times today


AttributeError: 'NoneType' object has no attribute 'text'

What do you notice takes the place when booking is not found?

We could use exception handling (`try`/`except` blocks) to resolve this. However, exceptions should only be used to handle rare or unexpected errors -- never for normal program flow.

In this case, we expect that some entries will be zero. So, we can just use an `if` statement that tests whether there are any `divs` present; if not, display `'ZERO'`. Here's a demo:

In [62]:
# if we find the element we want, we print it. Otherwise, we print 'ZERO'
for entry in html.find_all('div', {'class':'result content-section-list-row cf with-times'}):
    booking_tag = entry.find('div', {'class':'booking'})
    
    if booking_tag:
        print booking_tag.text
    else:
        print 'ZERO'

Booked 29 times today
Booked 181 times today
Booked 45 times today
Booked 69 times today
Booked 89 times today
Booked 283 times today
Booked 147 times today
Booked 178 times today
Booked 309 times today
ZERO
Booked 332 times today
Booked 30 times today
Booked 80 times today
Booked 53 times today
Booked 31 times today
Booked 56 times today
Booked 572 times today
Booked 42 times today
Booked 81 times today
Booked 30 times today
Booked 43 times today
Booked 29 times today
Booked 147 times today
Booked 71 times today
Booked 21 times today
Booked 195 times today
Booked 85 times today
Booked 43 times today
Booked 36 times today
Booked 29 times today
Booked 33 times today
Booked 37 times today
Booked 35 times today
Booked 206 times today
Booked 85 times today
Booked 34 times today
Booked 22 times today
Booked 51 times today
Booked 79 times today
Booked 101 times today
Booked 58 times today
Booked 29 times today
Booked 91 times today
Booked 78 times today
Booked 221 times today
Booked 265 time

After previously completing this, we observed that all other elements WILL be returned. This means we do not have to always handle these cases.

<a id="challenge-pandas"></a>
### Challenge: Use Pandas to create a DataFrame of bookings

However, the onus is on you to now put all the pieces together.

Loop through each entry. For each entry, grab the relevant information we want (name, location, price, bookings). Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [60]:
import pandas as pd

In [61]:
# I'm going to create my empty df first
dc_eats = pd.DataFrame(columns=["name","location","price","bookings"])

In [75]:
# Put code here that populates the DataFrame using Selenium and BeautifulSoup!

In [64]:
# check out our work
dc_eats.head()

,name,location,price,bookings
0,Harold Black,Capitol Hill,2,29
1,Rasika,Penn Quarter,3,181
2,Chez Billy Sud,Georgetown,2,45
3,Centrolina,Downtown,2,69
4,Ambar,Capitol Hill,2,89


Awesome! We succeeded.

<a id="selenium-typing"></a>
### Auto-typing using Selenium

Now, let's explore some of the other functionality of a webdriver. We've barely scratched the surface.

In [65]:
# we can send keys as well
# import
from selenium.webdriver.common.keys import Keys

In [66]:
# open Firefox
driver = webdriver.Firefox()
# visit Python
driver.get("http://www.python.org")
# verify we're in the right place
assert "Python" in driver.title

Let's try automatedly typing `pycon` in the search box and hitting the return key:

In [67]:
# find the search position
elem = driver.find_element_by_name("q")

# clear it
elem.clear()

# type in pycon
elem.send_keys("pycon")

# send those keys
elem.send_keys(Keys.RETURN)

In [68]:
# close
driver.close()

In [69]:
# all at once:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title

elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
#assert "No results found." not in driver.page_source
driver.close()

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html

### Summary

In this lesson, we used the Beautiful Soup library to locate elements on a website then scrape their text. We also used the Selenium headless browser to run JavaScript first before retrieving the page contents.